# 单票含客人自有税号，主要流程如下
### 1. 调整客人自有税号重量，至35kg/箱
### 2. 重复第一步，直至所有客人自有税号调整完毕
### 3. 计算包税目标重量（AIR WAY BILL GW - STEP 1 & 2 GW)
### 4. 调整包税部分重量至目标重量

##### notes, N.W. = 0.75 * G.W. may need to adjust 35kg/box, probably creative a random value between 30-35 or let user define a value

##### created on 15/01/2019

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
from random import randint

In [2]:
wb = xw.Book(r'C:\Users\Xiaodong\Desktop\包税程序设计\重量计算流程及相关文件\重量计算流程及相关文件\单独申报 ： DIDADI 104件原始以及调整过程.xls')

# Creat a summary sheet with some weight information

In [3]:
if wb.sheets[0].name != 'Summary':
    xw.sheets.add(name='Summary', before=wb.sheets[0])  
    wb.sheets[0].range('A1').value = 'Air Way Bill GW'
    wb.sheets[0].range('A2').value = '原始包税 GW'
    wb.sheets[0].range('A3').value = '客人自有税号 GW'
    wb.sheets[0].range('A4').value = '包税修改后 GW'
    wb.sheets[0].range('A1:A4').color = (0, 225, 0)

In [4]:
# Create a variable to store all adjusted 自有税号 total G.W.
gw_zysh = []

# 1.a Start adjust first 自有税号
# Creat a new workbook to output the results

In [12]:
wb2 = xw.Book()

sht1 = wb2.sheets.active

# Assign the source sheets with original data

In [13]:
shtA = wb.sheets[3]    #worksheet A
sht_summary = wb.sheets[0]  #worksheet summary

# Set a variable of target weight say 35kg/box

In [8]:
# tgt_wt_per_box = randint(30, 35)  # create a random integer between 30 - 35
tgt_wt_per_box = 35           # set the value to 35, choose one of two options available

# Export to DataFrame

In [14]:
df = shtA.range('A20:Q23').options(pd.DataFrame).value   # export to dataframe change A1: U143 according to individual case

tgt_wt = df['CTN'].sum() * tgt_wt_per_box   # workout target weight for this 自有税号

perc = tgt_wt / df['G.W.(KG)'].sum()    # wortkout percentage increase

df['ADJUSTED G.W.'] = round(df['G.W.(KG)'] * perc, 2)

df['ADJUSTED N.W.'] = round(df['ADJUSTED G.W.'] * 0.75, 2)

gw_zysh.append(df['ADJUSTED G.W.'].sum())

# After above step, write out to excel and save the excel, start next 客人自有税号

# ------------------------------------------------------------

# Once all 客人自有税号 adjustment are done, start below

### calculate 包税目标重量 = Airway Bill GW - 自有税号调整后总重量

In [16]:
wb.sheets['Summary'].range('B3').value = sum(gw_zysh)   # write value to Summary sheet 客人自有税号 GW

In [17]:
bs_tgt_gw = wb.sheets['Summary'].range('B1').value - sum(gw_zysh)   # bs_tgt_gw 包税目标重量

In [20]:
wb.sheets['Summary'].range('B4').value = bs_tgt_gw

# Assign the source sheet

In [21]:
shtB = wb.sheets[1]    

# Creat a new workbook to output the results

In [22]:
wb2 = xw.Book()

sht1 = wb2.sheets.active   # create a new workbook  'Book3 in current case'

# Export to DataFrame
## note, need to have a column of item weight available, in this example, copied shipping weight column from the other sheet.

In [23]:
df = shtB.range('A2:R101').options(pd.DataFrame).value   # export to dataframe change A1: U143 according to individual case

In [24]:
df['CALCULATED GW'] = df['QTY'] * df['SHIIPING WEIGHT']  # create a column 'calculated gw' based on qty * item_wt

In [25]:
cal_gw = round(df['CALCULATED GW'].sum(), 2)

In [83]:
# sht_summary.range('B2').value / cal_gw, workout the adjustment percentage

1.0306306044864912

In [26]:
df['ADJUSTED GW'] = round(df['CALCULATED GW'] * round(sht_summary.range('B2').value / cal_gw, 2), 2)

In [27]:
diff = round(df['ADJUSTED GW'].sum() - df['G.W.(KG)'].sum(), 2)

diff

-0.8

# Find the item only appear once, apply diff to that item's weight

In [28]:
df['COMMODITYNAME'].value_counts(ascending=True).index[0]  #find the items appearance, pick the first one by add .index[0] 

# df.loc[df['英文品名'] == df['英文品名'].value_counts(ascending=True).index[0]]['ADJUSTED GW'] + diff

df.loc[df['COMMODITYNAME'] == df['COMMODITYNAME'].value_counts(ascending=True).index[0], 'ADJUSTED GW'] -= diff

# now start adjusting sensitive HS CODE weight

In [29]:
df['HS CODE'] = df['HS CODE'].astype(str)   # convert column HS CODE from float to string

def check_HS(x):
    if x.startswith('42') | x.startswith('61') | x.startswith('62'):
        return 'Sensitive'
    else:
        return 'Safe'

df['SENSITIVITY']= df['HS CODE'].apply(check_HS)  # create a column to set sensitivity according to HS CODE

## define a function to decide adjustment value according to qty

In [30]:
def weight_setter(qty):   
    if qty <=50:
        return 0.075
    elif qty <= 100:
        return 0.085
    elif qty <=150:
        return 0.035
    elif qty <=200:
        return 0.03
    else:
        return 0.025

# adjust sensitive HS CODE weight

In [31]:
def adjust_wt_sense(row):
    # weight = row[10]
    sens = row[19]            #need to adjust the number to the column of SENSITIVITY
    qty = row[11]             #need to adjust the number to the column of QTY
    
    if sens == 'Sensitive':
        return qty * weight_setter(qty)

df['ADJ1_SENS'] = round(df.apply(adjust_wt_sense, axis = 1), 2)

df['ADJ1'] = df['ADJ1_SENS'].combine_first(df['ADJUSTED GW'])   # replace the none value with original number

# adjust safe HS CODE weight

In [32]:
wgt_diff = sht_summary.range('B4').value - df['ADJ1'].sum()

In [33]:
wgt_diff

-166.28999999999996

In [34]:
df_safe = df[df['SENSITIVITY'] == 'Safe']

In [35]:
qty_safe = df_safe['QTY'].sum()

In [36]:
adj2_val = wgt_diff / qty_safe

In [37]:
adj2_val

-0.03994475138121546

In [38]:
def adjust_wt_safe(row):
    weight = row[21]     # row adj1
    sens = row[19]   
    qty = row[11]
    
    if sens == 'Safe':
        return round(weight + qty * adj2_val, 2)

In [39]:
df['ADJ2_SAFE'] = df.apply(adjust_wt_safe, axis = 1)

In [40]:
df['ADJ2'] = df['ADJ2_SAFE'].combine_first(df['ADJ1'])   # replace the none value with original number

#  Same again, find the items only appear once, apply final diff to that item's weight

In [41]:
final_diff = round(sht_summary.range('B4').value - df['ADJ2'].sum(), 2)

# final_diff

df['COMMODITYNAME'].value_counts(ascending=True).index[1]  #find the items appearance, pick the first one by add .index[0] 

df.loc[df['COMMODITYNAME'] == df['COMMODITYNAME'].value_counts(ascending=True).index[1], 'ADJ2'] += final_diff

# In this special case, there is one minus weight, below code will solve problem, 
# by copying the weight from ADJ1, then adjust differenc eto another item

In [43]:
mask = df['ADJ2'] < 0

In [48]:
df.loc[mask, 'ADJ2' ] = df.loc[mask, 'ADJ1']

In [50]:
final_diff2 = round(sht_summary.range('B4').value - df['ADJ2'].sum(), 2)

# final_diff

df['COMMODITYNAME'].value_counts(ascending=True).index[1]  #find the items appearance, pick the first one by add .index[0] 

df.loc[df['COMMODITYNAME'] == df['COMMODITYNAME'].value_counts(ascending=True).index[2], 'ADJ2'] += final_diff2

In [46]:
final_diff

0.07

# experiment code below

In [181]:
df.loc[mask, 'ADJ2' ] = df.loc[mask, 'ADJ1']

In [152]:
df[df['ADJ2'] < 0]['ADJ2'] = 2.06

C:\Users\Xiaodong\Anaconda2\envs\Python3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
mask = df['ADJ2'] < 0

Series([], Name: ADJ2, dtype: float64)

In [151]:
df[df['ADJ2'] < 0]['ADJ1']

MARKS & NO
FBA15C7LQY09U000001    2.06
Name: ADJ1, dtype: float64

In [167]:
unique_index = pd.Index(mask)

In [170]:
type(unique_index).get_loc

pandas.core.indexes.base.Index

In [140]:
final_diff

0.07

In [51]:
gw_zysh

[349.98, 105.0]

In [52]:
sum(gw_zysh)

454.98

# write back to worksheet, execute below code whenever necessary

In [51]:
sht1.range('A1').value = df

# clear worksheet content

In [142]:
sht1.clear_contents()

In [30]:
randint(30, 35)

33

# Things need to be improved, source file column header and position need to be consistent, otherwise everytime need to adjust the header
# Shipping Item Weight need to be on the source sheet for calculation purpose